# Introduction to Access Control

We can now start using CrypTen to carry out private computations in some common use cases. In this tutorial, we will look at the first two use cases described in the Introduction, <i>Feature Aggregation</i> and <i>Data Augmentation</i>. In both use cases, we'll use a simple two-party setting and demonstrate how we can learn a linear SVM. In the process, we will see how access control works in CrypTen. We'll return to creating `CrypTensors` with the high-level `crypten.cryptensor` factory function, as we did in Tutorial 1.

As usual, we'll begin by importing the `crypten` and `torch` libraries, and initialize `crypten` with `crypten.init()`.

In [14]:
import crypten
import torch

crypten.init()

## Application 1: Feature Aggregation

In this scenario, two parties, Alice and Bob, each have a part of the features of the dataset. We'll use the MNIST data to demonstrate how Alice and Bob can learn without revealing protected information. For reference, the feature size of each example in the MNIST data is `28 x 28`, and there are 60000 examples in training data and 10000 examples in the test data. 

Let's assume Alice has the first `28 x 20` features and Bob has last `28 x 8` features. One way to think of this split is that Alice has the (roughly) top 2/3rds of each image, while Bob has the bottom 1/3rd of each image. We'll see how we can use CrypTen to learn over all `28 x 28` features (i.e., the entire image), while keeping each party's features private.

For ease of use, we have created a helper script `mnist_utils.py` that downloads the publicly available MNIST data, and splits the data as required. We will first use this script to create a dataset such that Alice has the first `28 x 20` features of each example, and Bob has last `28 x 8` features of each example.

In [15]:
%run ../examples/mnist_utils.py --option features

In [16]:
data_alice_train = crypten.load('/tmp/alice_train.pth', src=0)
print(data_alice_train.size())

data_alice_test = crypten.load('/tmp/alice_test.pth', src=0)
print(data_alice_test.size())

torch.Size([60000, 28, 20])
torch.Size([10000, 28, 20])


We'll illustrate how Alice and Bob learn privately in 4 steps: (a) loading the data, (b) encrypting the data, (c) constructing the encrypted training data, and (d) training privately. 

### Step (a): Loading the data

We'll begin by loading Alice's data and Bob's data. 

Before we do so, we need to understand a little more about how CrypTen is implemented. CrypTen runs a separate process for each party, but each process runs the identical (complete) program. We therefore need a mechanism to ensure that each process holds its data, and shares only the encrypted version with the other processes. 

As is standard in MPI programming, CrypTen uses a `rank` variable to identify the process (and thus the party). Let's assume Alice has the `rank` 0 process and Bob has the `rank` 1 process. When loading the data, we have to provide the `CrypTensor` with the source rank, i.e., rank of the process that would hold its data. This is provided through the `src` keyword in the `crypten.cryptensor` load function. 

Let's look at an example now. We'll load both parties' data, and then we'll look at the sizes of the data tensors they hold. Again, both processes run all the code, so both Alice and Bob will create both data tensors. However, the `src` keyword tells the process whether to load the data from the file, or to load an empty tensor of the appropriate shape (this is a requirement of `torch.distributed`, our communication backend).   

<i><small>(Technical note: Because Jupyter notebooks run only a single process, we simulate a multi-party world with the `@mpc.run_multiprocess` decorator.)</small></i>

In [17]:
import crypten.mpc as mpc
import crypten.communicator as comm

@mpc.run_multiprocess(world_size=2)
def load_private_data():
    
    #Load Alice's data with src=0 since Alice has the rank 0 process
    x_alice = crypten.load('/tmp/alice_train.pth', src=0)
    
    #Load Bob's data with src=1 since Bob has the rank 1 process
    x_bob = crypten.load('/tmp/bob_train.pth', src=1)
    
    #Check the sizes of the data for each process
    rank = comm.get().get_rank() #We can access the rank of the process through this function
    print(f"Rank {rank} Alice data size: {x_alice.size()}")
    print(f"Rank {rank} Bob data size: {x_bob.size()}")
    
z = load_private_data()


Rank 1 Alice data size: torch.Size([60000, 28, 20])
Rank 0 Alice data size: torch.Size([60000, 28, 20])
Rank 0 Bob data size: torch.Size([60000, 28, 8])
Rank 1 Bob data size: torch.Size([60000, 28, 8])


### Step (b): Encrypting the data

Next, we encrypt the data by creating `CrypTensors`, just as we did in Tutorial 1. But here there is one crucial difference: we have to provide the same `src` of the data to the `crypten.cryptensor` function that we did to the `crypten.load` function. In our example, when creating `CrypTensor` for Alice's data, we should use `src=0`; when creating `CrypTensor` for Bob's data, we should use `src=1`. 

<i><small>(Technical note: As we mentioned before, because Jupyter notebooks run only a single process, we simulate a multi-party world with the `@mpc.run_multiprocess` decorator. However, as a result, the variables loaded do not carry over from cell to cell as is customary in a notebook. Therefore, we reinitialize `x_alice` and `x_bob` in each cell simulating a multi-party world.)</small></i>

In [18]:
@mpc.run_multiprocess(world_size=2)
def construct_encrypted_data():

    #Load the data
    #Load Alice's data with src=0
    x_alice = crypten.load('/tmp/alice_train.pth', src=0)
    #Load Bob's data with src=1
    x_bob = crypten.load('/tmp/bob_train.pth', src=1)

    #Encrypt the data: 
    #Alice's data gets encrypted with src=0
    x_alice_enc = crypten.cryptensor(x_alice, src=0)
    #Bob's data gets encrypted with src=1
    x_bob_enc = crypten.cryptensor(x_bob, src=1)
    
z = construct_encrypted_data()

Note that both the rank 0 and the rank 1 process call the `crypten.cryptensor` on both `x_alice` and `x_bob` tensors. However, because of the `src` argument of `crypten.cryptensor`, the rank 0 process and the rank 1 process perform different actions on both `x_alice` and `x_bob`. For `x_alice`, the rank 0 process will construct secret shares of `x_alice`, and provide a share to the rank 1 process; `x_alice_enc` will contain Alice's secret share for process 0 and Bob's secret share for process 1. For `x_bob`, the rank 1 process will construct secret shares of `x_alice`, and provide a share to the rank 1 process; `x_bob_enc` will contain Alice's secret share for process 0 and Bob's secret share for process 1. 


### Step (c): Constructing the Encrypted Training Data
To use both Alice's features and Bob's features for training, we'll construct a tensor that concatenates both encrypted tensors. We'll do this with CrypTen's `cat` function, similar to `torch.cat`. This creates a new `CrypTensor`.

<i><small>(Technical note: Again, when using the `@mpc.run_multiprocess` decorator, the variables loaded do not carry over from cell to cell as is customary in a notebook. Therefore, we reinitialize `x_alice`, `x_bob`, `x_alice_enc` and `x_bob_enc` in the following cell.)</small></i>

In [19]:
@mpc.run_multiprocess(world_size=2)
def construct_and_combine_encrypted_data():
    x_alice = crypten.load('/tmp/alice_train.pth', src=0)
    x_bob = crypten.load('/tmp/bob_train.pth', src=1)

    x_alice_enc = crypten.cryptensor(x_alice, src=0)
    x_bob_enc = crypten.cryptensor(x_bob, src=1)
    
    rank = comm.get().get_rank()
    print(f"Rank: {rank} Size of Alice's encrypted data: {x_alice_enc.size()}") 
    print(f"Rank: {rank} Size of Bob's encrypted data: {x_bob_enc.size()}")
    print()

    #using crypten.cat to combine the feature sets
    x_combined_enc = crypten.cat([x_alice_enc, x_bob_enc], dim=2)
    
    print(f"Rank: {rank} Size of the combined data: {x_combined_enc.size()}")
    print(f"Rank: {rank} Combined data encrypted: {crypten.is_encrypted_tensor(x_combined_enc)}")
    
z = construct_and_combine_encrypted_data()

Rank: 1 Size of Alice's encrypted data: torch.Size([60000, 28, 20])
Rank: 1 Size of Bob's encrypted data: torch.Size([60000, 28, 8])

Rank: 0 Size of Alice's encrypted data: torch.Size([60000, 28, 20])
Rank: 0 Size of Bob's encrypted data: torch.Size([60000, 28, 8])

Rank: 1 Size of the combined data: torch.Size([60000, 28, 28])
Rank: 1 Combined data encrypted: True
Rank: 0 Size of the combined data: torch.Size([60000, 28, 28])
Rank: 0 Combined data encrypted: True


Note that we do not reveal any private information by doing so: process 0 will construct a tensor that concatenates Alice's shares of `x_alice_enc` and `x_bob_enc`, and process 1 will construct a tensor that concatenates Bob's shares of `x_alice_enc` and `x_bob_enc`. 

We can now use this data to train in CrypTen just as we would use plaintext data in PyTorch. 

### Step (d): Training with Encrypted Data 
We'll now use a linear SVM classifier to show how CrypTen can train on encrypted data. CrypTen implements all of the necessary operations required for this (and many other) learning algorithms to operate on encrypted tensors, so we can implement the learning in the same way as we would on plaintext tensors. 

The code below implements the learning algorithm in CrypTen. While each step is carried out on ```CrypTensors```, the learning algorithm looks just as it would in PyTorch! The only difference is that, in CrypTen, the learned weights and bias are ```CrypTensors```. If the plaintext versions of weights and bias are required, Alice and Bob will have to agree to decrypt them at the end of the training. 

In [20]:
# The following code is required for demonstrating the learning algorithm in our notebook. 
# As Jupyter notebooks run only a single process, Alice and Bob both need to encrypt with 
# src=0 in order for the remaining code to run. In a regular CrypTen implementation 
# (see the CrypTen examples folder), x_enc_bob would be encrypted with src=1 as shown in the cells above.
x_alice = crypten.load('/tmp/alice_train.pth', src=0)
x_alice_enc = crypten.cryptensor(x_alice, src=0)

#The following two lines would use src=1 when run outside a Jupyter notebook.
x_bob = crypten.load('/tmp/bob_train.pth', src=0)
x_bob_enc = crypten.cryptensor(x_bob, src=0) 

x_combined_enc = crypten.cat([x_alice_enc, x_bob_enc], dim=2)

In [21]:
#Load labels
label_train = crypten.load('/tmp/train_labels.pth')
label_test = crypten.load('/tmp/test_labels.pth')

# Modify the labels so that:
# all non-zero digits have class label 1.
# all zero digits have class label -1
label_train[label_train == 0] = -1
label_train[label_train != 0] = 1
label_test[label_test == 0] = -1
label_test[label_test != 0] = 1
 
#We'll use only the first 10k examples so it runs faster
data_enc = x_combined_enc[:10000,:,:]
labels = label_train[:10000]
examples = data_enc.size(0)

In [22]:
# Random initialization for linear svm
w_init = torch.randn(1, 28*28)
b_init = torch.randn(1)
 
# Turn all tensors into encrypted tensors
y_enc = crypten.cryptensor(labels)   
w_enc = crypten.cryptensor(w_init)
b_enc = crypten.cryptensor(b_init)

#define parameters: epoch and learning rate
epochs = 50
lr = 0.1
log_accuracy = True

x_flatten_enc = data_enc.flatten(start_dim=1)

for i in range(epochs):
        # Forward
        yhat = w_enc.matmul(x_flatten_enc.t()) + b_enc
        yhat = yhat.sign()

        yy = yhat * y_enc

        if log_accuracy and i%5 == 4:
            # Compute accuracy
            correct = (yy + 1).mul(0.5).sum()
            print("Epoch %d" % (i + 1))
            print(
                "--- Accuracy %.2f%%"
                % (correct.get_plain_text().float().div(examples).item() * 100)
            )
        # Backward
        loss_grad = y_enc * (yy - 1) * 0.5

        b_grad = loss_grad.sum()/examples
        w_grad = loss_grad.matmul(x_flatten_enc)/examples

        # Update
        w_enc = w_enc - w_grad * lr
        b_enc = b_enc - b_grad * lr

Epoch 5
--- Accuracy 86.25%
Epoch 10
--- Accuracy 94.56%
Epoch 15
--- Accuracy 96.56%
Epoch 20
--- Accuracy 97.51%
Epoch 25
--- Accuracy 98.03%
Epoch 30
--- Accuracy 98.37%
Epoch 35
--- Accuracy 98.59%
Epoch 40
--- Accuracy 98.75%
Epoch 45
--- Accuracy 98.87%
Epoch 50
--- Accuracy 99.02%


In [23]:
#Finally, we decrypt the weights
print("CrypTen weights:", w_enc.get_plain_text())
print("CrypTen bias:", b_enc.get_plain_text())

CrypTen weights: tensor([[ 5.7780e-01,  1.2894e+00, -8.7309e-01, -6.1832e-01,  1.1711e+00,
          4.5486e-02, -1.8327e+00,  1.8540e+00,  3.9236e-01, -8.5716e-01,
          1.1499e+00,  1.2826e+00, -6.0641e-01,  7.0221e-02, -3.0348e-01,
         -4.7008e-01, -2.2560e-01,  1.0942e+00,  8.7486e-01,  1.7632e+00,
         -1.1922e+00, -8.2465e-01, -1.3471e+00, -7.1735e-01,  1.9817e+00,
         -3.6327e-01,  1.1384e+00, -2.0858e+00,  1.0813e+00, -1.3672e+00,
         -2.1877e-01,  7.3682e-01,  1.1379e+00,  5.3615e-01, -1.2881e+00,
         -5.1572e-01, -1.8575e+00,  2.9251e-02,  5.3888e-01, -2.5453e+00,
         -9.8712e-01, -4.5172e-01, -1.1684e+00, -2.3492e+00,  2.2895e+00,
         -1.2922e+00,  4.8813e-01,  9.8215e-01,  4.1728e-01, -4.9069e-01,
          7.9715e-01, -1.7189e-01, -5.3986e-02,  1.6958e+00, -7.4800e-01,
          9.4496e-01, -1.1786e-01,  1.8239e+00,  6.6832e-01, -2.1777e+00,
         -7.7705e-01,  1.2912e-01, -5.0041e-01,  8.6090e-01, -1.6803e+00,
         -5.8594e-03,

In [24]:
# Let's examine our accuracy on the test data
w_final = w_enc.get_plain_text()
b_final = b_enc.get_plain_text()

#Let's load the test data. We'll load plaintext versions and get back torch tensors, since we are testing plaintext
test_alice = crypten.load('/tmp/alice_test.pth')
test_bob = crypten.load('/tmp/bob_test.pth')
test_complete = torch.cat([test_alice, test_bob], dim=2)
test_flattened = test_complete.flatten(start_dim=1)
targets = label_test

#compute output
output = w_final.matmul(test_flattened.t()) + b_final
output_sign = output.sign()

#compute accuracy of output
output_target = output_sign*targets
correct = (output_target + 1).mul(0.5).sum().float()
accuracy = correct/targets.size(0) * 100
print("Test Accuracy: %.2f%%" % accuracy.item())

Test Accuracy: 99.07%


Alternately, Alice and Bob may only need the labels of the test data in plaintext. In this situation, we would not need to decrypt `w_enc` and `b_enc`. Instead, we could encrypt the the test data, and use the encrypted classifier (i.e., with `w_enc` and `b_enc`) to classify the encrypted test data. The labels we get will be encrypted, and only these we would need to decrypt. The trained classifier itself remains encrypted.  

There is one final item to understand. As we did in the earlier tutorials, we have used `get_plain_text` to decrypt the `CrypTensors`. For this function to succeed, all the parties have to communicate their secret shares in order to carry out the decryption. Thus, the `CrypTensors` can only be decrypted if Alice and Bob agree to do so. 

## Application 2: Data Augmentation

Next, we'll show how we can use CrypTen in the <i>Data Augmentation</i> application. Here Alice and Bob each have some examples, and would like to learn a classifier over their combined examples. As before, Alice and Bob wish to keep their respective data private. 

The steps we take are very similar to the <i>Feature Aggregation</i> application: (a) initialize each process with its data and dummy input, (b) encrypt the data, (c) concatenate the data, and (d) learn on encrypted tensors. Indeed, the main difference comes in Step (c), where the concatenation of the `CrypTensors` is done along the batch dimension.

Let's walk through the first few steps to make this clear. We'll assume that because Alice and Bob each have part of the examples, they will also have only the corresponding part of the labels. Thus, we'll encrypt the labels and combine the encrypted labels as well.

In [25]:
#First, we'll use the mnist_utils.py script to split the public MNIST data appropriately for Alice and Bob
%run ../examples/mnist_utils.py --option data

In [26]:
@mpc.run_multiprocess(world_size=2)
def construct_and_combine_encrypted_data():

    #Step (a): Load each party's data into their process
    x_alice = crypten.load('/tmp/alice_train.pth', src=0)
    x_bob = crypten.load('/tmp/bob_train.pth', src=1)
    
    y_alice = crypten.load('/tmp/alice_train_labels.pth', src=0)
    y_bob = crypten.load('/tmp/bob_train_labels.pth', src=1)

        
    #Step (b): Encrypt the data
    x_alice_enc = crypten.cryptensor(x_alice, src=0)
    y_alice_enc = crypten.cryptensor(y_alice, src=0)

    x_bob_enc = crypten.cryptensor(x_bob, src=1)
    y_bob_enc = crypten.cryptensor(y_bob, src=1)
    
    rank = comm.get().get_rank()
    
    #Step (c): Create the combined encrypted data
    print(f"Rank {rank} Size of Alice's encrypted data:\n Examples: {x_alice_enc.size()} Labels: {y_alice_enc.size()}") 
    print(f"Rank {rank} Size of Bob's encrypted data:\n Examples: {x_bob_enc.size()} Labels: {y_bob_enc.size()}")
    print()

    #Combine the examples and labels: concatenate along batch dimension
    x_combined_enc = crypten.cat([x_alice_enc, x_bob_enc], dim=0)
    y_combined_enc = crypten.cat([y_alice_enc, y_bob_enc], dim=0)

    print(f"Rank {rank} Size of the combined data:\n Examples: {x_combined_enc.size()} Labels: {y_combined_enc.size()}")
    print(f"Rank {rank} Combined data:\n Examples encrypted: {crypten.is_encrypted_tensor(x_combined_enc)}." + 
            f" Labels encrypted: {crypten.is_encrypted_tensor(y_combined_enc)}.")
        
z = construct_and_combine_encrypted_data()

Rank 1 Size of Alice's encrypted data:
 Examples: torch.Size([43200, 28, 28]) Labels: torch.Size([43200])
Rank 1 Size of Bob's encrypted data:
 Examples: torch.Size([16800, 28, 28]) Labels: torch.Size([16800])

Rank 0 Size of Alice's encrypted data:
 Examples: torch.Size([43200, 28, 28]) Labels: torch.Size([43200])
Rank 0 Size of Bob's encrypted data:
 Examples: torch.Size([16800, 28, 28]) Labels: torch.Size([16800])

Rank 1 Size of the combined data:
 Examples: torch.Size([60000, 28, 28]) Labels: torch.Size([60000])
Rank 1 Combined data:
 Examples encrypted: True. Labels encrypted: True.
Rank 0 Size of the combined data:
 Examples: torch.Size([60000, 28, 28]) Labels: torch.Size([60000])
Rank 0 Combined data:
 Examples encrypted: True. Labels encrypted: True.


Step (c) contains only main difference from the <i>Feature Aggregation</i> application. Here we concatenated the data along the batch dimension (`dim 0`), while in <i>Feature Aggregation</i>, we used the feature dimension (`dim 1`). 

We can now train with this data exactly as we did earlier, in Step (d).

This completes our tutorial on access control in CrypTen in the context of two common applications.